In [2]:
import os
import logging
os.environ["CUDA_DEVICE_ORDER"]='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [3]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("training.log"),
        logging.StreamHandler()  
    ]
)
logger = logging.getLogger(__name__)

In [4]:
import torch
import argparse
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM
from mamba_trainer.data import DataModule
from mamba_trainer.data import LongRangeDataset
from mamba_trainer.trainer import MambaTrainer, GradientCallback
from datasets import load_dataset

/home/sapr7y/noisy_ssm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-05 23:06:05.638607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 23:06:05.657687: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 23:06:05.663521: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 23:06:05.678634: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binar

In [5]:
model = AutoModelForCausalLM.from_pretrained("state-spaces/mamba-130m-hf")
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token

optimizer = torch.optim.Adam(model.parameters(), amsgrad=True)

lora_config = LoraConfig(
    r=16,
    target_modules=["x_proj", "embeddings", "in_proj", "out_proj"],
    task_type="CAUSAL_LM",
    bias="none"
)

model = get_peft_model(model, lora_config)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MambaForCausalLM(
      (backbone): MambaModel(
        (embeddings): lora.Embedding(
          (base_layer): Embedding(50280, 768)
          (lora_dropout): ModuleDict(
            (default): Identity()
          )
          (lora_A): ModuleDict()
          (lora_B): ModuleDict()
          (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 16x50280 (cuda:0)])
          (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 768x16 (cuda:0)])
          (lora_magnitude_vector): ModuleDict()
        )
        (layers): ModuleList(
          (0-23): 24 x MambaBlock(
            (norm): MambaRMSNorm(768, eps=1e-05)
            (mixer): MambaMixer(
              (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
              (act): SiLU()
              (in_proj): lora.Linear(
                (base_la

In [7]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)



trainable params: 3,796,608 || all params: 132,931,968 || trainable%: 2.86


In [8]:
training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    output_dir="model",
    logging_dir="logs", 
    logging_steps=1,
    save_steps=1,
    report_to="none", 
)

In [11]:
from torch.utils.data import Subset
import numpy as np
grad_callback = GradientCallback()
data_module = DataModule(data_path="./data/basic_20-70/train.tsv", tokenizer=tokenizer)
dataset = data_module.dataset
for i in tqdm(range(20)):
    np.random.seed(None)
    ids = np.random.choice(len(dataset), size=4, replace=False)
    print(ids)
    subset = Subset(dataset, ids.tolist())
    #dataloader = DataLoader(subset, batch_size=4, collate_fn=data_module.data_collator, shuffle=True)
    for ds in subset:
        print(ds["input_ids"].shape)



    trainer = MambaTrainer(
        model=model,
        args=training_args,
        train_dataset=subset,
        tokenizer=tokenizer,
        optimizers=(optimizer, None),
        data_collator=data_module.data_collator,
        callbacks=[grad_callback]
    )

    trainer.train()

./data/basic_20-70/train.tsv


  0%|          | 0/20 [00:00<?, ?it/s]

[257 664 525 962]
torch.Size([47])
torch.Size([100])
torch.Size([91])
torch.Size([61])


2024-08-05 23:08:07,536 - mamba_trainer.trainer - INFO - Gradient Norm at step 0: 0.0


Step,Training Loss
1,4.103900


...

2024-08-05 23:08:27,926 - mamba_trainer.trainer - INFO - Training Loss at step 1: 4.6203
 95%|█████████▌| 19/20 [00:21<00:01,  1.06s/it]

[604 885 661 212]
torch.Size([77])
torch.Size([84])
torch.Size([84])
torch.Size([49])


2024-08-05 23:08:28,874 - mamba_trainer.trainer - INFO - Gradient Norm at step 0: 0.0


Step,Training Loss
1,3.598300


2024-08-05 23:08:28,960 - mamba_trainer.trainer - INFO - Training Loss at step 1: 3.5983
100%|██████████| 20/20 [00:22<00:00,  1.12s/it]
